<a href="https://colab.research.google.com/github/PadalaKavya/MajorProject/blob/base/Twitter_Hatespeech_Dataset_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt

from sklearn import model_selection, preprocessing, linear_model, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import ensemble
from sklearn.metrics import roc_auc_score, roc_curve
from xgboost import XGBClassifier


import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from textblob import Word
nltk.download('wordnet')

from termcolor import colored
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn import set_config
set_config(print_changed_only = False)

print(colored("\nLIBRARIES WERE SUCCESFULLY IMPORTED...", color = "green", attrs = ["dark", "bold"]))


LIBRARIES WERE SUCCESFULLY IMPORTED...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
train_set = pd.read_csv("train.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)

test_set = pd.read_csv("test.csv",
                   encoding = "utf-8",
                   engine = "python",
                   header = 0)

print(colored("\nDATASETS WERE SUCCESFULLY LOADED...", color = "green", attrs = ["dark", "bold"]))


DATASETS WERE SUCCESFULLY LOADED...


In [ ]:
train_set.head(n = 5)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
test_set.head(n = 5)

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [ ]:
print("Train set shape: {} and test set shape: {}".format(train_set.shape, test_set.shape))

Train set shape: (31962, 3) and test set shape: (17197, 2)


In [ ]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [ ]:
print("Totally there are {} duplicated values in train_set".format(train_set.duplicated().sum()))

Totally there are 0 duplicated values in train_set


In [ ]:
train_set.groupby("label").count()

,id,tweet
label,,
0,29720,29720
1,2242,2242


UPPERCASE COVERT TO LOWERCASE IN TWEETS

In [ ]:
train_set["tweet"] = train_set["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
test_set["tweet"] = test_set["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))

print(colored("\nCONVERTED SUCCESFULLY...", color = "green", attrs = ["dark", "bold"]))


CONVERTED SUCCESFULLY...


DELETE PUNCTUATION MARKS

In [ ]:
train_set["tweet"] = train_set["tweet"].str.replace('[^\w\s]','')
test_set["tweet"] = test_set["tweet"].str.replace('[^\w\s]','')

print(colored("\nDELETED SUCCESFULLY...", color = "green", attrs = ["dark", "bold"]))


DELETED SUCCESFULLY...


DELETE NUMBERS FROM TWEET COLUMN

In [ ]:
train_set['tweet'] = train_set['tweet'].str.replace('\d','')
test_set['tweet'] = test_set['tweet'].str.replace('\d','')

print(colored("\n NUMBERS DELETED SUCCESFULLY...", color = "green", attrs = ["dark", "bold"]))


 NUMBERS DELETED SUCCESFULLY...


DELETE STOPWORDS

In [ ]:
sw = stopwords.words("english")
train_set['tweet'] = train_set['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
test_set['tweet'] = test_set['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))

print(colored("\nSTOPWORDS DELETED SUCCESFULLY...", color = "green", attrs = ["dark", "bold"]))


STOPWORDS DELETED SUCCESFULLY...


In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:

  
train_set['tweet'] = train_set['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
test_set['tweet'] = test_set['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 

print(colored("\nLEMMATIZATION DONE SUCCESFULLY...", color = "green", attrs = ["dark", "bold"]))


LEMMATIZATION DONE SUCCESFULLY...


In [ ]:
train_set = train_set.drop("id", axis = 1)
test_set = test_set.drop("id", axis = 1)

print(colored("\n'ID' COLUMNS DROPPED SUCCESFULLY...", color = "green", attrs = ["dark", "bold"]))


'ID' COLUMNS DROPPED SUCCESFULLY...


In [ ]:
train_set.head(n = 10)

,label,tweet
0,0,user father dysfunctional selfish drag kid dys...
1,0,user user thanks lyft credit cant use cause do...
2,0,bihday majesty
3,0,model love u take u time urð ðððð ððð
4,0,factsguide society motivation
5,0,huge fan fare big talking leave chaos pay disp...
6,0,user camping tomorrow user user user user user...
7,0,next school year year examsð cant think ð scho...
8,0,love land allin cavs champion cleveland clevel...
9,0,user user welcome im gr


In [ ]:
test_set.head(n = 10)

,tweet
0,studiolife aislife requires passion dedication...
1,user white supremacist want everyone see new â...
2,safe way heal acne altwaystoheal healthy healing
3,hp cursed child book reservation already yes ð...
4,rd bihday amazing hilarious nephew eli ahmir u...
5,choose momtips
6,something inside dy ððâ eye ness smokeyeyes ti...
7,finishedtattooinkedinkloveitâï âïâïâïâï thanks...
8,user user user never understand dad left young...
9,delicious food lovelife capetown mannaepicure ...


In [ ]:
x = train_set["tweet"]
y = train_set["label"]

train_x, test_x, train_y, test_y = model_selection.train_test_split(x, y, test_size = 0.20, shuffle = True, random_state = 11)

print(colored("\nDIVIDED SUCCESFULLY...", color = "green", attrs = ["dark", "bold"]))


DIVIDED SUCCESFULLY...


TF-IDF VECTORIZATION

In [ ]:

vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features= 500000,lowercase=True)
vectorizer.fit(train_x)

x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

x_train_count.toarray()


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()

log_model = rfc.fit(x_train_count, train_y)
#y_pred = rfc.predict(x_test_count)

accuracy = model_selection.cross_val_score(log_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()
print('Test acc:',model_selection.cross_val_score(log_model,
                                           x_train_count,
                                           train_y,
                                           cv = 20).mean())
print(colored("\nRandomForest model ", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)

KeyboardInterrupt: ignored

In [ ]:

log = linear_model.LogisticRegression()

log_model = log.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(log_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

print(colored("\nLogistic regression model ", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)



Logistic regression model 
Accuracy ratio:  0.9391353046594985


In [ ]:

log = MultinomialNB()
y_pred = log.predict(X_test)
print('Test acc:',log.fit(test_y, y_pred))
log_model = log.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(log_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

print(colored("\nMultinomialNB model ", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)



MultinomialNB model 
Accuracy ratio:  0.9361260454002391


In [ ]:
from sklearn.tree import DecisionTreeClassifier
log = DecisionTreeClassifier()
log_model = log.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(log_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

print(colored("\nDecision Tree model, color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)



Decision Tree model with 'count-vectors' method
Accuracy ratio:  0.9379746970472777


In [ ]:

xgb = XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

print(colored("\nXGBoost model with 'count-vectors' method", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf_model = rf.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(rf_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

print(colored("\nRF model", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)

In [ ]:
from sklearn.svm import LinearSVC
sv = LinearSVC()
sv_model = sv.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(sv_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

print(colored("\nSVM model", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)


SVM model
Accuracy ratio:  0.9507040450588835


TF-IDF

In [ ]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
sentiment_model = Pipeline(
    steps=[
        (
            "TfidfVectorizer",TfidfVectorizer(ngram_range=(1,2), max_features= 500000,lowercase=True)
        ),
        (
            "naive_bayes", MultinomialNB()
       )
])

In [ ]:
sentiment_model.fit(train_x, train_y)

In [ ]:

predictions  = sentiment_model.predict(test_x)
accuracy = sentiment_model.score(test_x, test_y)
print(colored("\nNaive Bayes model with 'tf-idf' method", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)


In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
sentiment_model = Pipeline(
    steps=[
        (
            "TfidfVectorizer",TfidfVectorizer(ngram_range=(1,2), max_features= 500000,lowercase=True)
        ),
        (
            "linearsvc", LinearSVC()
       )
])

In [ ]:
sentiment_model.fit(train_x, train_y)
predictions  = sentiment_model.predict(test_x)
accuracy = sentiment_model.score(test_x, test_y)
print(colored("SVM model with 'tf-idf' method", color = "red", attrs = ["dark", "bold"]))
print(colored("Accuracy ratio: ", color = "red", attrs = ["dark", "bold"]), accuracy)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import nltk
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from sklearn.model_selection import train_test_split
from nltk.stem.wordnet import WordNetLemmatizer
import re
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Embedding, Flatten, Dense, Conv1D, MaxPool1D, MaxPooling1D
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import plotly.express as px
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
def define_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 100, input_length=max_length))
  model.add(Conv1D(filters=32, kernel_size=8, activation='relu'))
  model.add(MaxPool1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(10, activation='relu', kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))
  model.add(Dense(6, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
max_words=10000
tokenizer=Tokenizer(max_words)
tokenizer.fit_on_texts(train_set)
sequence_train=tokenizer.texts_to_sequences(train_set)
sequence_test=tokenizer.texts_to_sequences(test_set)

In [ ]:
word2vec=tokenizer.word_index
V=len(word2vec)
print('dataset has %s number of independent tokens' %V)

dataset has 2 number of independent tokens


In [ ]:
from tensorflow.keras.layers import Input,Conv1D,MaxPooling1D,Dense,GlobalMaxPooling1D,Embedding
from tensorflow.keras.models import Model

In [ ]:
T=train_set.shape[1]
T

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
import matplotlib.pyplot as plt
%matplotlib inline
file=pd.read_csv('train.csv')
file

In [ ]:
import re
import string
def remove_punct(text):
    text_nopunct = ''
    text_nopunct = re.sub('['+string.punctuation+']', '', str(text))
    return text_nopunct
file['Cleaned_Tweets'] = file['tweet'].apply(lambda x: remove_punct(x))
file.head()

In [ ]:
from nltk import word_tokenize
import nltk
nltk.download('punkt')

In [ ]:
tokens = file['Cleaned_Tweets'].map(word_tokenize)
file['Tokenized'] = [' '.join(t) for t in tokens]
tokens

In [ ]:
def lower_token(tokens): 
    return [word.lower() for word in tokens]    
    
tokens_lowered=[lower_token(t) for t in tokens]
file['tokens_lowered']=tokens_lowered
file.head()

In [ ]:
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords

stoplist=stopwords.words('english')
def removeStopWords(tokens): 
    return [w for w in tokens if w not in stoplist]
def removenumeric(tokens):
    return [w for w in tokens if not w.isdigit()]
filtered_words=[removeStopWords(t) for t in tokens_lowered]
filtered_words=[removenumeric(t) for t in filtered_words]
file['Text_Final'] = [' '.join(t) for t in filtered_words]
file['Filtered_words'] = filtered_words
file.head(8)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(file['Text_Final'], file['label'], shuffle=True, test_size=0.2)

# find the length of the largest sentence in training data
training_sentence_lengths = [len(tokens) for tokens in xtrain]
print("Max sentence length in train is %s" % max(training_sentence_lengths))
max_len=max(training_sentence_lengths)
# find the length of the largest sentence in testing data
test_sentence_lengths = [len(tokens) for tokens in xtest]
print("Max sentence length in test is %s" % max(test_sentence_lengths))

In [ ]:
from tensorflow.keras.preprocessing import text, sequence 

max_words = 10000
tokenizer = text.Tokenizer(num_words = max_words)

# create the vocabulary by fitting on x_train text
tokenizer.fit_on_texts(xtrain)
# generate the sequence of tokens
xtrain_seq = tokenizer.texts_to_sequences(xtrain)
xtest_seq = tokenizer.texts_to_sequences(xtest)

# pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq)
xtest_pad = sequence.pad_sequences(xtest_seq)
word_index = tokenizer.word_index
len(xtrain_pad[6])#len=142

In [ ]:
!wget http://downloads.cs.stanford.edu/nlp/data/glove.twitter.27B.zip

In [ ]:
!unzip glove.twitter.27B

In [ ]:
#load the Glove Vectors
import zipfile
embedding_vectors = {}
with open('glove.twitter.27B.100d.txt','r',encoding='utf-8') as glove_file:
    for row in glove_file:
        values = row.split(' ')
        word = values[0]
        weights = np.asarray([float(val) for val in values[1:]])
        embedding_vectors[word] = weights
#print(f"Size of vocabulary in GloVe: {len(embedding_vectors)}")   
#len(embedding_vectors)

In [ ]:
emb_dim = 100
if max_words is not None: 
    vocab_len = max_words 
else:
    vocab_len = len(word_index)+1
embedding_matrix = np.zeros((vocab_len, emb_dim))
oov_words = []
for word, idx in word_index.items():
    if idx < vocab_len:
        embedding_vector = embedding_vectors.get(word)
        if embedding_vector is not None:
            embedding_matrix[idx] = embedding_vector
        else:
            oov_words.append(word)
#print some of the out of vocabulary words
print(oov_words[0:50])

In [ ]:
embedding_matrix[2]

In [ ]:
#model LSTM
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout
from tensorflow.keras.models import Sequential

lstm_model = Sequential()
lstm_model.add(Embedding(vocab_len, emb_dim, trainable = False, weights=[embedding_matrix]))
lstm_model.add(LSTM(128, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(64, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(32, return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(16, return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(32, activation='relu'))
lstm_model.add(Dropout(0.1))
lstm_model.add(Dense(1, activation = 'sigmoid'))
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(lstm_model.summary())

In [ ]:
batch_size = 64
epochs  = 5
lstm = lstm_model.fit(xtrain_pad, ytrain, validation_data=(xtest_pad, ytest), batch_size = batch_size, epochs = epochs)

In [ ]:
train_lstm_results=lstm_model.evaluate(xtrain_pad, ytrain, verbose=0, batch_size=64)
test_lstm_results = lstm_model.evaluate(xtest_pad, ytest, verbose=0, batch_size=64)
print(f'Train accuracy: {train_lstm_results[1]*100:0.2f}')
print(f'Test accuracy: {test_lstm_results[1]*100:0.2f}')

In [ ]:
#model BLSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout
from tensorflow.keras.models import Sequential
blstm_model=Sequential()
blstm_model.add(Embedding(vocab_len, emb_dim, trainable = False, weights=[embedding_matrix]))
blstm_model.add(Bidirectional(LSTM(128, return_sequences=True)))
blstm_model.add(Dropout(0.2))
blstm_model.add(Bidirectional(LSTM(64, return_sequences=True)))
blstm_model.add(Dropout(0.2))
blstm_model.add(Bidirectional(LSTM(32, return_sequences=True)))
blstm_model.add(Dropout(0.2))
blstm_model.add(Bidirectional(LSTM(16, return_sequences=False)))
blstm_model.add(Dropout(0.2))
blstm_model.add(Dense(32, activation='relu'))
blstm_model.add(Dropout(0.1))
blstm_model.add(Dense(1,activation='sigmoid'))
blstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(blstm_model.summary())

In [ ]:
batch_size = 64
epochs=5
blstm=blstm_model.fit(xtrain_pad, ytrain, validation_data=(xtest_pad, ytest), batch_size = batch_size, epochs = epochs)

In [ ]:
train_blstm_results=blstm_model.evaluate(xtrain_pad, ytrain, verbose=0, batch_size=64)
test_blstm_results=blstm_model.evaluate(xtest_pad, ytest, verbose=0, batch_size=64)
print(f'Train accuracy: {train_blstm_results[1]*100:0.2f}')
print(f'Test accuracy: {test_blstm_results[1]*100:0.2f}')

In [ ]:
class Sentiment:

    def preprocessing(string):
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
        
        string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
        string = re.sub(r"\'s", " \'s", string)
        string = re.sub(r"\'ve", " \'ve", string)
        string = re.sub(r"n\'t", " n\'t", string)
        string = re.sub(r"\'re", " \'re", string)
        string = re.sub(r"\'d", " \'d", string)
        string = re.sub(r"\'ll", " \'ll", string)
        string = re.sub(r",", " , ", string)
        string = re.sub(r"!", " ! ", string)
        string = re.sub(r"\(", " \( ", string)
        string = re.sub(r"\)", " \) ", string)
        string = re.sub(r"\?", " \? ", string)
        string = re.sub(r"\s{2,}", " ", string)
        string = emoji_pattern.sub(r'', string)
        return string.strip().lower()
    
    
    def batch_iter(data, batch_data, num_epochs):
        
        data = np.array(data)
        data_size = len(data)
        num_batch_per_epoch = int(len(data) - 1/ batch_size )+ 1
        for epoch in range(num_epoch):
            if shuffle:
                shuffle_indices = np.random.permutation(np.arange(data_size))
                shuffled_data = data[shuffle_indices]
            else:
                shuffled_data = data
            for batch_num in range(num_batches_per_epoch):
                start_index = batch_num * batch_size
                end_index = min((batch_num + 1) * batch_size, data_size)
                yield shuffled_data[start_index:end_index]
                
                
    def load_data_and_labels(data, max_features, maximum_len):
        
        from keras.preprocessing.text import Tokenizer
        from tensorflow.keras.preprocessing.sequence import pad_sequences
        from keras.utils import to_categorical
        
        train = data.sample(frac = 1).reset_index(drop=True)
        
        

        X = data['tweet'].apply(lambda x: Sentiment.preprocessing(x))
        Y = to_categorical(data['label'].values)
        tokenizer = Tokenizer(num_words=max_features)
        tokenizer.fit_on_texts(list(X))
        
        X = tokenizer.texts_to_sequences(X)
        X = pad_sequences(X, maxlen = maximum_len)
        
        
        return X, Y

In [ ]:
train_data = pd.read_csv('train.csv')

In [ ]:
X_train, Y_train = Sentiment.load_data_and_labels(train_data, max_features, maximum_len)
print(X_train.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size = 0.2, random_state = 42)

In [ ]:
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
maximum_len = 300
max_features = 10000
model = Sequential()

# Input / Embdedding
model.add(Embedding(max_features, 150, input_length=maximum_len))

# CNN
model.add(SpatialDropout1D(0.2))

model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

# Output layer
model.add(Dense(2, activation='sigmoid'))

In [ ]:
epochs = 5
batch_size = 32

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
train_cnn_results=model.evaluate(X_train, Y_train, verbose=1, batch_size=32)
test_cnn_results=model.evaluate(X_test, Y_test, verbose=1, batch_size=32)
print(f'Train accuracy: {train_cnn_results[1]*100:0.2f}')
print(f'Test accuracy: {test_cnn_results[1]*100:0.2f}')

In [ ]:
#To find F1-score, precision, recall, accuracy
from sklearn.metrics import classification_report
print(classification_report(X_train, Y_train, verbose=1, batch_size=32))

In [ ]:
#to have the predict results